In [ ]:
# Prerequisites: images in ./bear_images/ folder, test.csv in the ./ folder.

In [4]:
!git clone https://github.com/ultralytics/yolov5
!git clone https://github.com/timolai-andrievich/find-a-bear-yolov5n-weights

fatal: destination path 'yolov5' already exists and is not an empty directory.
fatal: destination path 'find-a-bear-yolov5n-weights' already exists and is not an empty directory.


In [5]:
%pip install -qr yolov5/requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
from PIL import Image
import os

In [7]:
!python yolov5/detect.py --weights find-a-bear-yolov5n-weights/yolov5n.pt --save-conf --save-txt --conf-thres 0 --max-det 1 --nosave --source bear_images

detect: weights=['find-a-bear-yolov5n-weights/yolov5n.pt'], source=bear_images, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.0, iou_thres=0.45, max_det=1, device=, view_img=False, save_txt=True, save_conf=True, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  v7.0-128-gb96f35c Python-3.9.13 torch-2.0.0+cu117 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs
image 1/489 C:\Projects\find-a-bear\bear_images\image_1.jpeg: 640x640 1 bear, 15.0ms
image 2/489 C:\Projects\find-a-bear\bear_images\image_10.jpeg: 640x640 1 bear, 16.0ms
image 3/489 C:\Projects\find-a-bear\bear_images\image_100.png: 160x640 1 bear, 69.0ms
image 4/489 C:\Projects\find-a-bear\bear_images\image_101.jpeg: 640x640

In [8]:
data = {col: [] for col in 'file_name x1 y1 x2 y2 confidence'.split()}

In [16]:
def process_file(file_name):
    img = Image.open(f'bear_images/{file_name}')
    basename, _ext = os.path.splitext(file_name)
    with open(f'yolov5/runs/detect/exp/labels/{basename}.txt', 'r') as file:
        _cls, x, y, w, h, confidence = map(float, file.read().split())
    x1, y1, x2, y2 = x - w / 2, y - h / 2, x + w / 2, y + w / 2
    x1, y1, x2, y2 = map(int, (x1 * img.width, y1 * img.height, x2 * img.width, y2 * img.height))
    data['file_name'].append(file_name)
    data['x1'].append(x1)
    data['y1'].append(y1)
    data['x2'].append(x2)
    data['y2'].append(y2)
    data['confidence'].append(confidence)

In [17]:
test_df = pd.read_csv('test.csv')

In [18]:
test_df.file_name.apply(process_file)

0      None
1      None
2      None
3      None
4      None
       ... 
144    None
145    None
146    None
147    None
148    None
Name: file_name, Length: 149, dtype: object

In [19]:
submission = pd.DataFrame(data)

In [20]:
submission.loc[submission['confidence'] < .1, 'x1 y1 x2 y2 confidence'.split()] = 0
submission.loc[submission['confidence'] >= .1, 'confidence'] = 1.

In [21]:
submission.to_csv('submission.csv', index=False)